# NARX


In [1]:
# @title Lendo a planilha

import numpy as np
import pandas as pd

df = pd.read_csv("../export/dados.csv", index_col=0)

ciclos = df.index.values
t_feed = df["t_feed"].values
t_rinse = df["t_rinse"].values
t_blow = df["t_blow"].values
t_purge = df["t_purge"].values

purity_H2 = df["purity_H2"].values
H2_CO_ratio = df["H2_CO_ratio"].values
purity_CO2 = df["purity_CO2"].values
recovery_CO2 = df["recovery_CO2"].values
productivity = df["productivity"].values


model_name = "NARX"


In [2]:
# Colocando dados em desvio

y = np.array(
    [
        purity_H2,
        H2_CO_ratio,
        purity_CO2,
        recovery_CO2,
        productivity,
    ]
).T
u = np.array([t_feed, t_rinse, t_blow, t_purge]).T

# ciclo de referência
ref_idx = 1
y_ref = np.tile(y[ref_idx, :], (len(y), 1))
u_ref = np.tile(u[ref_idx, :], (len(u), 1))

# --- Calcular desvios ---
y_desvio = y - y_ref
u_desvio = u - u_ref


In [3]:
from lib.utils import benchmark


def gerar_NARX(y, u):
    y_prev = y[:-1, :]
    u_prev = u[:-1, :]
    u_now = u[1:, :]

    # Termos lineares
    X_lin = np.hstack([y_prev, u_prev, u_now])
    # Termos quadráticos (não-lineares)
    X_quad = np.hstack([y_prev**2, u_prev**2, u_now**2])
    # Termos de interação
    X_inter = np.hstack(
        [
            (y_prev[:, :, None] @ u_prev[:, None, :]).reshape(len(y_prev), -1),
            (y_prev[:, :, None] @ u_now[:, None, :]).reshape(len(y_prev), -1),
        ]
    )

    # Matriz de regressão final
    X = np.hstack([X_lin, X_quad, X_inter])

    # Saídas futuras
    Y = y[1:, :]

    # Estimando parâmetros via mínimos quadrados
    theta = np.linalg.lstsq(X, Y)[0]
    return theta


def inferir_NARX(theta, y, u):
    N = len(y)
    y_narx = np.zeros_like(y)
    y_narx[0] = y[0]  # inicializa com o primeiro valor real

    for k in range(1, N):
        yk = y_narx[k - 1]
        uk_prev = u[k - 1]
        uk_now = u[k]

        phi_lin = np.concatenate([yk, uk_prev, uk_now])
        phi_quad = np.concatenate([yk**2, uk_prev**2, uk_now**2])
        phi_inter = np.concatenate(
            [
                (yk[:, None] @ uk_prev[None, :]).ravel(),
                (yk[:, None] @ uk_now[None, :]).ravel(),
            ]
        )

        phi = np.concatenate([phi_lin, phi_quad, phi_inter])

        y_narx[k] = phi @ theta
    return y_narx


theta = gerar_NARX(y_desvio, u_desvio)
print("Parâmetros estimados:\n", theta.shape)

y_narx = inferir_NARX(theta, y_desvio, u_desvio)
y_narx = y_narx + y_ref  # Convertendo o valor desvio em variavel de engenharia

# Benchmarking
benchmark(inferir_NARX, model_name, theta, y_desvio, u_desvio)


Parâmetros estimados:
 (66, 5)
Benchmark file already exists. Skipping benchmark.


In [4]:
# Plotando resultados
from lib.plot import plot_comparison

mse = np.mean((y - y_narx) ** 2)
print(f"MSE {model_name}:", mse)

plot_comparison(ciclos, y, y_narx, model_name)


MSE NARX: 0.026272637405137536
